# MNIST Classifier with HuoguoML

This short notebook shows how to use HuoguoML to deploy and manage a service. This includes:

1. Building and training a neural network to classify MNIST images
2. Create a HuoguoML service

## Requirements

We will be using Tensorflow 2 and HuoguoML. Update `pip` and install packages:

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install huoguoml

Import all packages:

In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

## Building and training a neural network to classify MNIST images

### Data

First we load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Further, the samples are converted from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Create a neural network
A two layer fully connected neural network, that is trained with cross entropy loss and optimized with adam model:

In [ ]:
# define two sets of inputs
inputA = tf.keras.Input(shape=(28, 28))
inputB = tf.keras.Input(shape=(28, 28))
# the first branch operates on the first input
x = tf.keras.layers.Flatten()(inputA)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(10, activation="relu")(x)
x = tf.keras.Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = tf.keras.layers.Flatten()(inputB)
y = tf.keras.layers.Dense(128, activation='relu')(y)
y = tf.keras.layers.Dense(10, activation="relu")(y)
y = tf.keras.Model(inputs=inputB, outputs=y)

# our model will accept the inputs of the two branches and
# then output a single value
model = tf.keras.Model(inputs=[x.input, y.input], outputs=[x.output, y.output])
model.summary()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train, evaluate and export neural network

Train neural network

In [ ]:
model.fit(x=[x_train, x_train], y=[y_train, y_train], epochs=5)

Evaluate neural network -the image classifier should have a accuracy of ~98% on this dataset.

In [ ]:
model.evaluate([x_test, x_test],  [y_test, y_test], verbose=2)

Export neural network

In [ ]:
model_dir = "./model_dual"
model.save(model_dir)

In [ ]:
!saved_model_cli show --dir ./model_dual --all

## Create a HuoguoML service

In [ ]:
f.structured_input_signature

In [ ]:
import huoguoml

huoguoml.__version__
huoguoml.tensorflow.log_model(tf_saved_model_dir="./model_dual", 
                              tf_meta_graph_tags="serve",
                              tf_signature_def_key="serving_default",
                              artifact_dir="./artifact")